In [67]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import RMSprop
from keras import optimizers
import sys
from keras.callbacks import LambdaCallback
import random
import matplotlib.pyplot as plt


import numpy as np
import os
import time

In [68]:
tweet_data=pd.read_csv('../Load_Tweets/data/tweet_data.csv')

In [69]:
tweet_text = tweet_data['TEXT']

In [70]:
tweet_text_all = tweet_data['TEXT'].str.cat(sep=' ')

In [71]:
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn\'t she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders. Hillary &amp; CNN FRAUD! Thank you Texas! If you haven\'t registered to VOTE- today is your last day. Go to: [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY. You can register now at: [URL] and get out to… [URL] DON\'T LET HER FOOL US AGAIN. [URL] Crooked\'s State Dept gave special attention to "Friends of Bill" after the Haiti Earthquake. Unbelieva

In [72]:
print ('Length of text: {} characters'.format(len(tweet_text_all)))

Length of text: 1513769 characters


In [73]:
# Take a look at the first 250 characters in text
print(tweet_text_all[:250])

PAY TO PLAY POLITICS. #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been t


In [74]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [75]:
import re
tweet_text_all=re.sub('(\w)\.(\w)', '\\1. \\2', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4})(\w)', ' \\1 \\2 \\3', tweet_text_all)
tweet_text_all=re.sub('(\w)(\.{2,4}) ', '\\1 \\2 ',tweet_text_all) #fdj... fdj ...
tweet_text_all=re.sub(' (\.{2,6})(\w)', ' \\1 \\2',tweet_text_all) #...fdj ... fdj
tweet_text_all=re.sub('[)(]', '', tweet_text_all)


In [76]:
# list_of_words=pd.Series(tweet_text_all.split())
# print ('Length of text: {} words'.format(len(list_of_words)))
# word_vocab=set(list_of_words)
# print ('{} unique words'.format(len(word_vocab)))

In [77]:
tweet_text_all=re.sub('([^.])\.([^.])', '\\1 EOS\\2', tweet_text_all) 
# tweet_text_all = tweet_text_all.replace(".", ' <EOS>')
tweet_text_all[0:1000]

'PAY TO PLAY POLITICS EOS #CrookedHillary [URL] Very little pick-up by the dishonest media of incredible information provided by WikiLeaks EOS So dishonest! Rigged system! Crooked Hillary Clinton likes to talk about the things she will do but she has been there for 30 years - why didn\'t she do them? Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again EOS Lets get out &amp;… [URL] Join me Thursday in Florida &amp; Ohio!West Palm Beach, FL at noon:[URL]Cincinnati, OH this 7:30pm:[URL] Wow, @CNN Town Hall questions were given to Crooked Hillary Clinton in advance of big debates against Bernie Sanders EOS Hillary &amp; CNN FRAUD! Thank you Texas! If you haven\'t registered to VOTE- today is your last day EOS Go to: [URL] &amp; get ou… [URL] VOTER REGISTRATION DEADLINES TODAY EOS You can register now at: [URL] and get out to… [URL] DON\'T LET HER FOOL US AGAIN EOS [URL] Crooked\'s State Dept gave special attention to "Friends of Bill" after the Haiti 

In [78]:
list_of_words=pd.Series(tweet_text_all.split())
print ('Length of text: {} words'.format(len(list_of_words)))

Length of text: 266327 words


In [79]:
#number of unique words
word_vocab=sorted(set(list_of_words))
print ('{} unique words'.format(len(word_vocab)))

27784 unique words


In [80]:
# The unique characters in the file
vocab = sorted(set(tweet_text_all))
print ('{} unique characters'.format(len(vocab)))

367 unique characters


In [81]:
#replace apostrophes in dictionary keys 
for i in range (len(list_of_words)):
    list_of_words[i]=list_of_words[i].replace("‘", '').replace("’", '').replace("'", '')

In [82]:
#see how many words after apostrophe deletion
word_vocab=sorted(set(list_of_words))
print ('{} unique words'.format(len(word_vocab)))

27390 unique words


In [83]:
word_vocab.append('UNK')

In [84]:
word_vocab[-1]

'UNK'

In [85]:
#create forward and reverse word index
word2idx = dict((c, i) for i, c in enumerate(word_vocab, 1))
# idx2word = dict((i, c) for i, c in enumerate(word_vocab,1 ))
idx2word= np.asarray(word_vocab)

text_as_int = np.array([word2idx[w] for w in list_of_words])

In [86]:
print('{')
for word,_ in zip(word2idx, range(100)):
    print('  {:4s}: {:3d},'.format(repr(word), word2idx[word]))
print('  ...\n}')

{
  'comprehensive,': 16347,
  'pointed': 22380,
  'product': 22750,
  'Cook': 6088,
  'on!': 21725,
  'topic!': 25495,
  'Pag': 10948,
  'before"': 15269,
  'Fed': 7604,
  'nasty': 21380,
  'nearly': 21402,
  '#BlacksForTrump': 889,
  'Corruption': 6115,
  'peo…': 22178,
  'crowd-': 16801,
  'act,': 14433,
  'admonished': 14501,
  '1929': 2174,
  '@dbongino:': 3941,
  'tax!': 25092,
  'itself,': 20134,
  'Aviv': 4833,
  'TOLD': 12969,
  '"incident"': 705,
  '"@SirHatchporch:': 239,
  'baby!"': 15126,
  'lover': 20754,
  '+2': 1905,
  'FL': 7468,
  'country': 16638,
  'tolerate': 25456,
  'Aides:': 4484,
  'questions?!?': 22967,
  'Poland,': 11205,
  'FIV…': 7465,
  '14': 2111,
  'mining': 21143,
  '“His': 26914,
  'Hawaii!': 8332,
  'observing': 21626,
  '83%': 2781,
  'repressed': 23464,
  'Frankenstien': 7800,
  'information,': 19876,
  '#OttoWarmbier': 1374,
  'fatalities': 18348,
  'long-suffering': 20681,
  '#failing@nytimes': 1738,
  'EASTERN': 6818,
  'WON,': 13856,
  '5,481,73

In [87]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- words mapped to int ---- > {}'.format(repr(' '.join(list_of_words[:13])), text_as_int[:13]))

'PAY TO PLAY POLITICS EOS #CrookedHillary [URL] Very little pick-up by the dishonest' ---- words mapped to int ---- > [10831 12959 10858 10868  6856   936 14295 13692 20618 22267 15722 25244
 17358]


In [88]:
idx2word

array(['!', '!!', '!!!', ..., '🦃', '\U0010fc00', 'UNK'], dtype='<U123')

In [89]:
# The maximum length sentence we want for a single input in characters
seq_length = 20
examples_per_epoch = len(list_of_words)//seq_length

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(5):
  print(idx2word[i.numpy()])

PAY-FOR-PLAY
TODAY
PLEDGE?
POLL
EOS!


In [90]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2word[item.numpy()])))  


'PAY-FOR-PLAY TODAY PLEDGE? POLL EOS! #CrookedHillary! [URL]#AZPrimary Very, little, picked by, the#USWomensOpen, dishonest! media! of! incredible! information! provides by, WikiLeaks: EOS!'
'So, dishonest, Rigged! system, Crooked-USED! Hillary! Clinton! likewise to, talk! about! the#USWomensOpen, things! she, will! do! but, she, has, been! there!'
'for! 30% years! -&gt; why! didnt! she, do! theme Thank-You you! Florida-tomorrow a..... MOVEMENT! that! has, never! been! seen! before! and#PoliceWeek'
'will! never! be! seen! again! EOS! Level get! out! &gt;[URL] [URL]#AZPrimary Joined me! Thursday! in! Florida! &amp;, Ohio![URL] Palmer, Bean FL,'
'at! noon:[URL]Warren, OH, this! 7:35 Wow,the @CNN! Towns, Hall- questions! were, gives to, Crooked-USED! Hillary! Clinton! in! advanced of! big! debates!'


In [91]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [92]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'PAY-FOR-PLAY TODAY PLEDGE? POLL EOS! #CrookedHillary! [URL]#AZPrimary Very, little, picked by, the#USWomensOpen, dishonest! media! of! incredible! information! provides by, WikiLeaks:'
Target data: 'TODAY PLEDGE? POLL EOS! #CrookedHillary! [URL]#AZPrimary Very, little, picked by, the#USWomensOpen, dishonest! media! of! incredible! information! provides by, WikiLeaks: EOS!'


In [93]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2word[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2word[target_idx])))

Step    0
  input: 10831 ('PAY-FOR-PLAY')
  expected output: 12959 ('TODAY')
Step    1
  input: 12959 ('TODAY')
  expected output: 10858 ('PLEDGE?')
Step    2
  input: 10858 ('PLEDGE?')
  expected output: 10868 ('POLL')
Step    3
  input: 10868 ('POLL')
  expected output: 6856 ('EOS!')
Step    4
  input: 6856 ('EOS!')
  expected output: 936 ('#CrookedHillary!')


In [94]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 20), (64, 20)), types: (tf.int64, tf.int64)>

In [95]:
embeddings_index = {}
f = open('../word_embeding/glove.twitter.27B.25d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [96]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)+1

# The embedding dimension 
embedding_dim = 25

# Number of RNN units
rnn_units = 1024
embedding_matrix = np.zeros((vocab_size, embedding_dim)) 

for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [97]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
  print ("on the GPU BABY!")
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

on the GPU BABY!


In [98]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):

    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                batch_input_shape=[batch_size, None,],
                                trainable=False),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model

In [136]:
model = build_model(
  vocab_size = vocab_size, 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [137]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 25)            684800    
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (64, None, 1024)          3228672   
_________________________________________________________________
dense_4 (Dense)              (64, None, 27392)         28076800  
Total params: 31,990,272
Trainable params: 31,305,472
Non-trainable params: 684,800
_________________________________________________________________


In [138]:
for input_example_batch, target_example_batch in dataset.take(1): 
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 20, 27392) # (batch_size, sequence_length, vocab_size)


In [139]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [140]:
sampled_indices

array([20489, 17980,  2711, 18558,  8742,  9334, 10906,  3191,   331,
        3018, 24018, 24437, 19105,  9127, 25388, 23141, 20723, 23621,
       15669, 25223])

In [141]:
print("Input: \n", repr(" ".join(idx2word[[input_example_batch[0]]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices])))

Input: 
 'had, no! impact! on! Liu Rocketry Man, EOS! Hariri to, believe! his! people,” and#PoliceWeek the#USWomensOpen, military- puts up! with! loaded'

Next Char Predictions: 
 'lets ethic! 73, flooding, Incredible, LIED PRIDE @GrahamLedger "@drgoodspine: @CynthiaLummis![URL] senator somewhat g… KNOCK tight reason? losses, revolutionary buoy tha'


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """Entry point for launching an IPython kernel.


In [142]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 20, 27392)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       10.218005


In [143]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [144]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [145]:
EPOCHS=10

In [146]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/10
208/208 [==============================] - 35s 169ms/step - loss: 8.2152
Epoch 2/10
208/208 [==============================] - 33s 159ms/step - loss: 7.9354
Epoch 3/10
208/208 [==============================] - 33s 159ms/step - loss: 7.8418
Epoch 4/10
208/208 [==============================] - 31s 149ms/step - loss: 7.7338
Epoch 5/10
208/208 [==============================] - 31s 148ms/step - loss: 7.6131
Epoch 6/10
208/208 [==============================] - 31s 148ms/step - loss: 7.4025
Epoch 7/10
208/208 [==============================] - 31s 148ms/step - loss: 7.2757
Epoch 8/10
208/208 [==============================] - 31s 149ms/step - loss: 7.1776
Epoch 9/10
208/208 [==============================] - 31s 148ms/step - loss: 7.0955
Epoch 10/10
208/208 [==============================] - 31s 148ms/step - loss: 7.0227


In [110]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [147]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [148]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 25)             684800    
_________________________________________________________________
cu_dnngru_5 (CuDNNGRU)       (1, None, 1024)           3228672   
_________________________________________________________________
dense_5 (Dense)              (1, None, 27392)          28076800  
Total params: 31,990,272
Trainable params: 31,305,472
Non-trainable params: 684,800
_________________________________________________________________


In [149]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of words to generate
  num_generate = 30

  # Converting our start string to numbers (vectorizing) 
  if start_string!='' or ' ':
      for word in start_string.split():
        if word in word2idx:
          input_eval = [word2idx[word]]
          input_eval = tf.expand_dims(input_eval, 0)
        else:
            input_eval= [word2idx['UNK']]
            input_eval = tf.expand_dims(input_eval, 0)
      else: 
        input_eval= [word2idx['UNK']]
        input_eval = tf.expand_dims(input_eval, 0)
      
    

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      
      text_generated.append(idx2word[predicted_id])

  return (start_string + ' '.join(text_generated))

In [151]:
print(generate_text(model, start_string=u"remain"))

remainall OCare Death” 28, we, B! Homeland Superstar! 620,000 energy! released, expansion action-it of! Classified, HISTORY Islamist and#PoliceWeek TODAY!#MakeAmericaGreatAgain more! knows! Pence, responses heart re-enter @KenCalvert EOS! Saudi reasons, class,


In [61]:
print(generate_text("yeah", 10))

ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (1, 10, 25)